# Document's Features

#### · Display raw data from cluster

#### · Display raw data and located peaks recorded in cluster 

#### · Find peaks of recorded raw data with the most updated version of the algorithm 

#### · Important value to consider in the functions.py document is the value between 0 and 1 in the line                 
#### if (f_derivative < 0) and (b_derivative > 0) and y[i]>0.6*peak_average: 
#### which is multiplying peak_average


In [1]:
from functions import *
import pandas as pd
import numpy as np
import math 
import plotly.graph_objects as go
import sys
import matplotlib.pyplot as plt
from numpy import matrix
from scipy.signal import find_peaks
from scipy import interpolate
import datetime
import itertools as IT
import time
from scipy.interpolate import lagrange
from datetime import datetime
import plotly.io as pio

In [2]:
# Extracting data from files with sampling frequency of 250 HZ

In [12]:
df_time = pd.read_csv('time.csv') 
df_values = pd.read_csv('values.csv')
column_name_time = '0'
column_name_values = '0'
time_array_string = df_time[column_name_time].values
#Negating the values is returning them to the original form, just to make it consistent with the code in the hub
values_array = -df_values[column_name_values].to_numpy()

In [22]:
time_str = [f[11:len(f)] for f in time_array_string]
t = [datetime.strptime(sample_time, '%H:%M:%S.%f') for sample_time in time_str]
time_in_mili = np.array([time_sample.minute*60*1000 +time_sample.second*1000+ time_sample.microsecond/1000 for time_sample in t]) - (t[0].minute*60*1000 +t[0].second*1000 + t[0].microsecond/1000)

In [19]:
f = time_str[0]
f

'19:00:00.003'

In [23]:
time_in_mili

array([0.000000e+00, 4.000000e+00, 8.000000e+00, ..., 3.599986e+06,
       3.599990e+06, 3.599994e+06])

In [5]:
def time_in_format(time_array): 
    mins = ((time_array/1000)/60).astype(int);
    sec = ((time_array/1000)%60).astype(int);
    mili = ((time_array - (mins*60*1000 + sec*1000))*100).astype(int)/100 #*100.astype(int)/100 is to round to two decimals
    time_format = [str(minutes)+':'+str(seconds)+':'+str(milis) for minutes, seconds, milis in zip(mins,sec,mili)]

    return time_format

## Classes

In [6]:
#bs_r_window = baseline removal window;     #fs = sampling frequency                                      #trans_width = transition width of bandpass 
#numtaps = size of filter                   # mwi_windoow = moving window integration window


class sample_all_included():
    
    first_time = True
    
    def __init__(self,signal_values,min_min,min_sec,max_min,max_sec,seconds_per_label,inverted, bs_r_window,band_start,band_stop,fs,trans_width,numtaps, 
                 mwi_window, test_segments,spline,res):


        self.min_mins = min_min
        self.min_secs = min_sec
        self.max_mins = max_min
        self.max_secs = max_sec
        self.seconds_per_label = seconds_per_label
        
        min_time = int(((min_min*60*1000) + min_sec*1000)/4); max_time = int(((max_min*60*1000) + max_sec*1000)/4)+1; 
        
        self.values = -signal_values[min_time:max_time]
        self.time = time_in_mili[min_time:max_time]
        self.peaks,_ = find_peaks(self.values,distance = 100)
        

        
        self.tick_values = []
        x = 0
        while x < self.time[-1]:
            self.tick_values = np.append(self.tick_values,x)
            x = 1000*self.seconds_per_label + x
        
        self.time_ticks = time_in_format(self.tick_values) 




        #Filters' Parameters
        self.first_bs_r_window = bs_r_window
        self.first_band = [band_start, band_stop]
        self.first_mwi_window = mwi_window
        self.bs_r_window = bs_r_window
        self.trans_width = trans_width
        self.numtaps = numtaps
        self.fs = fs
        self.band = [band_start,band_stop]
        self.edges = [0, self.band[0] - trans_width, self.band[0], self.band[1], self.band[1] + self.trans_width, 0.5*fs]
        self.test_segments = test_segments
        self.segment_duration = 0.8
        self.spline = spline 
        self.res = res
         
        
        #Bandpass
        self.taps = signal.remez(self.numtaps, self.edges, [0, 1, 0], fs=self.fs)
        self.bp_x_in = np.append(self.prepro_data,np.zeros(len(self.taps)))
        self.bp_y = np.zeros(len(self.prepro_data)+len(self.taps))
        
        for j in range(0,len(self.prepro_data)+len(self.taps)): 
            sum = 0
            for k in range(0,len(self.taps)):
                sum = (self.bp_x_in[j-k]*self.taps[k]) + sum
            self.bp_y[j] = sum
            
        self.bp_y=np.delete(self.bp_y,[np.arange(0,int(len(self.taps)/2))])
        self.bp_x = self.time[0:len(self.bp_y)]

        
        #derivative
        self.der_y = np.zeros(len(self.bp_y))
        self.der_x = self.bp_x
        self.der_x_in = np.append(self.bp_y,np.zeros(2))

        for i in range(0,len(self.bp_y)):
            self.der_y[i] = (1/8) * (-(self.der_x_in[i-2]) - (2*self.der_x_in[i-1]) + (2*self.der_x_in[i+1]) + (self.der_x_in[i+2]))
        
        #squaring
        self.sq_y = (self.der_y) ** 2
        self.sq_x = self.der_x
        
        #Moving Window Integration
        self.mwi_window = mwi_window

        
        
    
    @property
    def bs_r_window(self): 
        return self._bs_r_window

    @bs_r_window.setter
    def bs_r_window(self,new_window_value):
        self._bs_r_window = new_window_value
        self.bs_re, self.cut_pts = moving_average(self._bs_r_window,self.values)
        self.prepro_data = self.values[0:len(self.time)-self.cut_pts] - self.bs_re
        self.prepro_x = self.time[0:len(self.time)-self.cut_pts]
        
        if (new_window_value != self.first_bs_r_window): 
            self.first_bs_r_window = 300
            self.bs_re, self.cut_pts = moving_average(self._bs_r_window,self.values)
            self.prepro_data = self.values[0:len(self.time)-self.cut_pts] - self.bs_re
            self.prepro_x = self.time[0:len(self.time)-self.cut_pts]
            
            self.edges = [0, self.band[0] - self.trans_width, self.band[0], self.band[1], self.band[1] + self.trans_width, 0.5*self.fs]
            self.taps = signal.remez(self.numtaps, self.edges, [0, 1, 0], fs=self.fs)
            self.bp_x_in = np.append(self.prepro_data,np.zeros(len(self.taps)))
            self.bp_y = np.zeros(len(self.prepro_data)+len(self.taps))
            
            for j in range(0,len(self.prepro_data)+len(self.taps)): 
                sum = 0
                for k in range(0,len(self.taps)):
                    sum = (self.bp_x_in[j-k]*self.taps[k]) + sum
                self.bp_y[j] = sum

            self.bp_y=np.delete(self.bp_y,[np.arange(0,int(len(self.taps)/2))])
            self.bp_x = self.time[0:len(self.bp_y)]
            
            #Derivative when bandpass is updated
            self.der_y = np.zeros(len(self.bp_y))
            self.der_x_in = np.append(self.bp_y,np.zeros(2))
            for i in range(0,len(self.bp_y)):
                self.der_y[i] = (1/8) * (-(self.der_x_in[i-2]) - (2*self.der_x_in[i-1]) + (2*self.der_x_in[i+1]) + (self.der_x_in[i+2]))

            self.der_x = self.bp_x

            #Squaring when bandpass is updatd
            self.sq_y = self.der_y ** 2
            self.sq_x = self.der_x

            #Movig window integgration when bandpass is updated 
            self.mwi_y = dif_eq_window_integration(self.sq_y,self.mwi_window,False)
            self.mwi_x = self.der_x

            #Fiducial points when bandpass is updated
            self.fiducial_points = find_fiducial_point(self.mwi_y,self.mwi_x,self.test_segments,self.segment_duration,self.mwi_window)
            self.max_x, self.max_y, self.peaks_x, self.peaks_y, self.interpolated_peak_x, self.interpolated_peak_y,self.max_ids = interpolation_spline(
                self.fiducial_points, self.time, self.values, self.spline, self.res)

            
        
    @property
    def band(self):
        return self._band
    
    @band.setter
    def band(self,new_band):
        self.band_start = new_band[0]; self.band_stop = new_band[1]
        self._band = [new_band[0],new_band[1]]
        if (new_band != self.first_band): 
            self.first_band = 0.01
            self.edges = [0, self._band[0] - self.trans_width, self._band[0], self._band[1], self._band[1] + self.trans_width, 0.5*self.fs]
            self.taps = signal.remez(self.numtaps, self.edges, [0, 1, 0], fs=self.fs)
            self.bp_x_in = np.append(self.prepro_data,np.zeros(len(self.taps)))
            self.bp_y = np.zeros(len(self.prepro_data)+len(self.taps))

            for j in range(0,len(self.prepro_data)+len(self.taps)): 
                sum = 0
                for k in range(0,len(self.taps)):
                    sum = (self.bp_x_in[j-k]*self.taps[k]) + sum
                self.bp_y[j] = sum

            self.bp_y=np.delete(self.bp_y,[np.arange(0,int(len(self.taps)/2))])
            self.bp_x = self.time[0:len(self.bp_y)]


            #Derivative when bandpass is updated
            self.der_y = np.zeros(len(self.bp_y))
            self.der_x_in = np.append(self.bp_y,np.zeros(2))
            for i in range(0,len(self.bp_y)):
                self.der_y[i] = (1/8) * (-(self.der_x_in[i-2]) - (2*self.der_x_in[i-1]) + (2*self.der_x_in[i+1]) + (self.der_x_in[i+2]))

            self.der_x = self.bp_x

            #Squaring when bandpass is updatd
            self.sq_y = self.der_y ** 2
            self.sq_x = self.der_x

            #Movig window integgration when bandpass is updated 
            self.mwi_y = dif_eq_window_integration(self.sq_y,self.mwi_window,False)
            self.mwi_x = self.der_x

            #Fiducial points when bandpass is updated
            self.fiducial_points = find_fiducial_point(self.mwi_y,self.mwi_x,self.test_segments,self.segment_duration,self.mwi_window)
            self.max_x, self.max_y, self.peaks_x, self.peaks_y, self.interpolated_peak_x, self.interpolated_peak_y,self.max_ids = interpolation_spline(
                self.fiducial_points, self.time, self.values, self.spline, self.res)
                    
    @property
    def mwi_window(self):
        return self._mwi_window
    
    @mwi_window.setter
    def mwi_window(self,new_window):
        self._mwi_window = new_window
        self.mwi_y = dif_eq_window_integration(self.sq_y,self.mwi_window,False)
        self.mwi_x = self.der_x
        self.fiducial_points = find_fiducial_point(self.mwi_y,self.mwi_x,self.test_segments,self.segment_duration,self.mwi_window)
        self.max_x, self.max_y, self.peaks_x, self.peaks_y, self.interpolated_peak_x, self.interpolated_peak_y,self.max_ids = interpolation_spline(
                self.fiducial_points, self.time, self.values, self.spline, self.res)
    
    

        
    
        #self.prepro_fig = go.Figure()
        #self.prepro_fig.add_trace(go.Scatter(line=dict(color='orange'),x = self.prepro_x, y = self.prepro_data),showlegend = show_legend)



In [7]:
"""
class sample: 
    def __init__(self,min_mins,min_secs,max_mins,max_secs,seconds_per_label,show_legend, legend,find_the_peaks):
        
        self.min_mins = min_mins
        self.min_secs = min_secs
        self.max_mins = max_mins
        self.max_secs = max_secs
        self.seconds_per_label = seconds_per_label
        
        min_time = int(((min_mins*60*1000) + min_secs*1000)/4); max_time = int(((max_mins*60*1000) + max_secs*1000)/4)+1; 
        
        self.values = -values_array[min_time:max_time]
        self.time = time_in_mili[min_time:max_time]
        self.peaks,_ = find_peaks(self.values,distance = 100)
        
        
            
        self.tick_values = []
        x = 0
        while x < self.time[-1]:
            self.tick_values = np.append(self.tick_values,x)
            x = 1000*self.seconds_per_label + x

        self.time_ticks = time_in_format(self.tick_values) 

        self.fig = go.Figure()

        self.fig.add_trace(
            go.Scatter(mode='lines',x=self.time, y=self.values,
                        name =legend,showlegend=show_legend))
        
        if find_the_peaks:
            self.peaks, _ = find_peaks(self.values, distance=100)
            self.fig.add_trace(
                      go.Scatter(mode='markers',x=self.time[self.peaks], y=self.values[self.peaks],showlegend=False))
"""


"\nclass sample: \n    def __init__(self,min_mins,min_secs,max_mins,max_secs,seconds_per_label,show_legend, legend,find_the_peaks):\n        \n        self.min_mins = min_mins\n        self.min_secs = min_secs\n        self.max_mins = max_mins\n        self.max_secs = max_secs\n        self.seconds_per_label = seconds_per_label\n        \n        min_time = int(((min_mins*60*1000) + min_secs*1000)/4); max_time = int(((max_mins*60*1000) + max_secs*1000)/4)+1; \n        \n        self.values = -values_array[min_time:max_time]\n        self.time = time_in_mili[min_time:max_time]\n        self.peaks,_ = find_peaks(self.values,distance = 100)\n        \n        \n            \n        self.tick_values = []\n        x = 0\n        while x < self.time[-1]:\n            self.tick_values = np.append(self.tick_values,x)\n            x = 1000*self.seconds_per_label + x\n\n        self.time_ticks = time_in_format(self.tick_values) \n\n        self.fig = go.Figure()\n\n        self.fig.add_trace(\n

In [8]:
# The function sample takes as arguments an interval of time in the first 4 arguments: 0,0,3,0 means the time interval starts 
# at minute 0 second 0 and finishes at minute 3 second 0 
# The 5th arggument is the seconds per label, in this case the x axis will display a 10 second scale. 
# Lastly if wether the legend should be display or not. 
"""
sample1 = sample(0,0,3,0,10,True,'Raw Data',True)
fig = sample1.fig
fig.update_layout(xaxis = dict(tickmode = 'array', tickvals = sample1.tick_values, ticktext = sample1.time_ticks),autosize=False,width=1250,height=600)
"""



"\nsample1 = sample(0,0,3,0,10,True,'Raw Data',True)\nfig = sample1.fig\nfig.update_layout(xaxis = dict(tickmode = 'array', tickvals = sample1.tick_values, ticktext = sample1.time_ticks),autosize=False,width=1250,height=600)\n"

### Getting a piece of signal with all included class

In [9]:
min_min_t = 20; min_sec_t = 0; max_min_t= 25; max_sec_t = 0; seconds_per_label_t = 10; inverted_t = False;   bs_r_window_t = 6; 
band_start_t = 3; band_stop_t = 100; fs_t= 250;  trans_width_t = 2.88; numtaps_t = 152; show_legend_t = False; mwi_window_t = 25 ;
test_segments_t =3;      spline_t = 4;       res_t = 5000;

sample_test = sample_all_included(values_array,min_min_t,min_sec_t,max_min_t,max_sec_t,seconds_per_label_t,inverted_t, bs_r_window_t,band_start_t,band_stop_t,fs_t,
                       trans_width_t,numtaps_t, mwi_window_t, test_segments_t, spline_t,res_t)

IndexError: index -1 is out of bounds for axis 0 with size 0

In [ ]:

fig_raw = go.Figure()
fig_raw.add_trace(go.Scatter(x = sample_test.time,y = sample_test.values,showlegend=False))
fig_raw.update_layout(xaxis = dict(tickmode = 'array', tickvals = sample_test.tick_values, ticktext = sample_test.time_ticks),autosize=False,width=1250,height=600);


In [ ]:
x_range = [600000,617000]

# Filtering the signal step by step

In [ ]:
fig_step_filt = make_subplots(rows=5, cols=1, subplot_titles=('Baseline Removal','Bandpass', 'Derivative','Squaring','Moving Window Integration'))
fig_step_filt.add_trace(go.Scatter(x = sample_test.prepro_x,y = sample_test.prepro_data,showlegend=False),row=1, col=1)
fig_step_filt.add_trace(go.Scatter(line = dict(color = 'orange'), x = sample_test.bp_x,y = sample_test.bp_y,showlegend=False),row=2, col=1)
fig_step_filt.add_trace(go.Scatter(line = dict(color = 'red'), x = sample_test.der_x,y = sample_test.der_y,showlegend=False),row=3, col=1)
fig_step_filt.add_trace(go.Scatter(line = dict(color = 'magenta'), x = sample_test.sq_x,y = sample_test.sq_y,showlegend=False),row=4, col=1)
fig_step_filt.add_trace(go.Scatter(line = dict(color = 'purple'), x = sample_test.mwi_x,y = sample_test.mwi_y,showlegend=False),row=5, col=1)



fig_step_filt.update_layout(xaxis_range=x_range, xaxis2_range=x_range,xaxis3_range=x_range,xaxis4_range=x_range,xaxis5_range=x_range,
                            yaxis_range=[-0.13,0.12], yaxis2_range=[-0.15,0.12], yaxis3_range=[-0.06,0.06], yaxis4_range=[-0.0001,0.0032],yaxis5_range=[-0.0001,0.017])
fig_step_filt.update_layout(height=1500,width = 1200)


fig_step_filt.update_layout(xaxis = dict(tickmode = 'array', tickvals = sample_test.tick_values, ticktext = sample_test.time_ticks),
                    xaxis2 = dict(tickmode = 'array', tickvals = sample_test.tick_values, ticktext = sample_test.time_ticks),
                    xaxis3 = dict(tickmode = 'array', tickvals = sample_test.tick_values, ticktext = sample_test.time_ticks),
                    xaxis4 = dict(tickmode = 'array', tickvals = sample_test.tick_values, ticktext = sample_test.time_ticks),
                    xaxis5 = dict(tickmode = 'array', tickvals = sample_test.tick_values, ticktext = sample_test.time_ticks));



## Identifying fiducial points in moving window integration

In [ ]:
fig_fiducial_mwi = go.Figure()

fig_fiducial_mwi.add_trace(go.Scatter(x = sample_test.mwi_x,y = sample_test.mwi_y,showlegend=False))
fig_fiducial_mwi.add_trace(go.Scatter(marker=dict(color='red'),mode ='markers',x = sample_test.mwi_x[sample_test.fiducial_points],
                                      y = sample_test.mwi_y[sample_test.fiducial_points],showlegend=False))
fig_fiducial_mwi.update_layout(xaxis = dict(tickmode = 'array', tickvals = sample_test.tick_values, ticktext = sample_test.time_ticks), autosize=False,width=1400,
                  height=600,showlegend=False,title="Moving Window Integration + Fiducial Points ", xaxis_title="Time", yaxis_title= 'Signal', font=dict(family="Avenir",size=14,color="Black"))

fig_fiducial_mwi.update_layout(xaxis_range=x_range);

In [ ]:
pio.write_image(fig_step_filt, 'step_by_step.jpg', format='jpeg')

#### Fiducial points in raw signal

In [ ]:
fig_fiducial_raw = go.Figure()

fig_fiducial_raw.add_trace(go.Scatter(x = sample_test.time,y = sample_test.values,showlegend=False))
fig_fiducial_raw.add_trace(go.Scatter(marker=dict(color='red'),mode ='markers',x = sample_test.time[sample_test.fiducial_points],
                                      y = sample_test.values[sample_test.fiducial_points],showlegend=False))

fig_fiducial_raw.update_layout(xaxis = dict(tickmode = 'array', tickvals = sample_test.tick_values, ticktext = sample_test.time_ticks), autosize=False,width=1400,
                  height=600,showlegend=False,title="Raw Data + Fiducial Points ", xaxis_title="Time", yaxis_title= 'Signal', font=dict(family="Avenir",size=14,color="Black"));



## Going from fiducial to peaks

### Fiducial points aren't always the exact peak but they are at least near them. After Fiducial points are identified the maximum points around it are searched. 

In [ ]:
fig_peaks_raw = go.Figure()

fig_peaks_raw.add_trace(go.Scatter(x = sample_test.time,y = sample_test.values,showlegend=False))
fig_peaks_raw.add_trace(go.Scatter(mode = 'markers', x = sample_test.max_x,y = sample_test.max_y,showlegend=False))


fig_peaks_raw.update_layout(xaxis = dict(tickmode = 'array', tickvals = sample_test.tick_values, ticktext = sample_test.time_ticks), autosize=False,width=1400,
                  height=600,showlegend=False,title="Raw Data + Peaks", xaxis_title="Time", yaxis_title= 'Signal', font=dict(family="Avenir",size=14,color="Black"))

#fig_peaks_raw.update_layout(xaxis_range=[69900,80000])

# Next step is to implement the frequency argument in the class

## Cluster Sample

In [24]:
cluster_sample = raw_sample_(1,30,3,0,1)
fig_raw = cluster_sample.fig
fig_raw.update_layout(xaxis = dict(tickmode = 'array', tickvals = cluster_sample.ticks_values, ticktext = cluster_sample.time_format_axis),autosize=False,width=1400,
                  height=600,showlegend=False,title="Raw Data ", xaxis_title="Time", yaxis_title= 'Signal', 
                  font=dict(family="Avenir",size=14,color="Black"));

## Pan Tompkins

In [25]:
# Sample One: sample_test

In [28]:
min_min_t = 0; min_sec_t = 0; max_min_t= 20; max_sec_t = 0; seconds_per_label_t = 15; inverted_t = False;   bs_r_window_t = 6; 
band_start_t = 3; band_stop_t = 100; fs_t= 250;  trans_width_t = 2.88; numtaps_t = 152; show_legend_t = False; mwi_window_t = 25 ;
test_segments_t =3;      spline_t = 4;       res_t = 5000;

sample_test = sample_all_included(min_min_t,min_sec_t,max_min_t,max_sec_t,seconds_per_label_t,inverted_t, bs_r_window_t,band_start_t,band_stop_t,fs_t,
                       trans_width_t,numtaps_t, mwi_window_t, test_segments_t, spline_t,res_t)

/home/shared/braulio/ecg_algorithm/functions.py:232: RuntimeWarning:

invalid value encountered in sqrt

/home/shared/braulio/ecg_algorithm/functions.py:233: RuntimeWarning:

invalid value encountered in sqrt



In [22]:
# Sample Two: sample_test_dif_wl    this sample will have different window length for the mean average subtraction

In [23]:
min_min_t = 1; min_sec_t = 30; max_min_t= 3; max_sec_t = 0; seconds_per_label_t = 1; inverted_t = False;   bs_r_window_t = 10; 
band_start_t = 3; band_stop_t = 100; fs_t= 250;  trans_width_t = 2.88; numtaps_t = 152; show_legend_t = False; mwi_window_t = 25 ;
test_segments_t =3;            spline_t = 4;       res_t = 5000;

sample_test_dif_wl = sample_all_included(min_min_t,min_sec_t,max_min_t,max_sec_t,seconds_per_label_t,inverted_t, bs_r_window_t,band_start_t,band_stop_t,fs_t,
                       trans_width_t,numtaps_t, mwi_window_t, test_segments_t, spline_t,res_t)

In [29]:
fig_raw = make_subplots(rows=3, cols=1, subplot_titles=("Peaks ", 'Mean Subtraction', 'Moving Window Integration'))

fig_raw.add_trace(go.Scatter(line = dict(color = 'blue'), x = sample_test.time,y = sample_test.values,showlegend=False),row=1, col=1)
fig_raw.add_trace(go.Scatter(mode = 'markers', x = sample_test.max_x,y = sample_test.max_y,showlegend=False),row=1, col=1)
fig_raw.add_trace(go.Scatter(x = sample_test.prepro_x,y = sample_test.prepro_data,showlegend=False),row=2,col=1)
fig_raw.add_trace(go.Scatter(line=dict(color='#0a4345'),x = sample_test.mwi_x,y = sample_test.mwi_y,showlegend=False),row=3,col=1)
fig_raw.add_trace(go.Scatter(marker=dict(color='red'),mode ='markers',x = sample_test.mwi_x[sample_test.fiducial_points],y = sample_test.mwi_y[sample_test.fiducial_points],
                              showlegend=False),row=3,col=1)



fig_raw.update_layout(xaxis = dict(tickmode = 'array', tickvals = sample_test.ticks_values, ticktext = sample_test.time_format_axis),
                    xaxis2 = dict(tickmode = 'array', tickvals = sample_test.ticks_values, ticktext = sample_test.time_format_axis),
                    xaxis3 = dict(tickmode = 'array', tickvals = sample_test.ticks_values, ticktext = sample_test.time_format_axis))

fig_raw.update_layout(xaxis_range=[79000,85000], xaxis2_range=[79000,85000],xaxis3_range=[79000,85000],yaxis_range=[-1.5,1.3],
                       yaxis2_range=[-0.4,0.4], yaxis3_range=[-0.001,0.05])
fig_raw.update_layout(height=1200,width = 1200);

In [31]:
#fig_raw.show()

## Mising Peaks

In [26]:
missing_pt_p, missing_cluster_p = find_missing_peaks(sample_test.max_x, cluster_sample.pks_time, True)


In [27]:
fig_c_vs_s =go.Figure()

fig_c_vs_s.add_trace(go.Scatter(line= dict(color='blue'),x=cluster_sample.time, y=cluster_sample.values,name='Raw Data',showlegend=True))
    

fig_c_vs_s.add_trace(go.Scatter(mode='markers',marker=dict(size=12),x=cluster_sample.pks_time, y= cluster_sample.pks_values, name = 'Cluster', 
                           showlegend=True,))

fig_c_vs_s.add_trace(go.Scatter(mode='markers',marker=dict(size=10,color='#21B626'), x = sample_test.max_x, 
                          y = sample_test.max_y, name = 'Current Version', showlegend=True))

fig_c_vs_s.add_trace(go.Scatter(mode='markers',marker=dict(size=7,color='orange'), x = sample_test.max_x[missing_cluster_p], 
                           y= sample_test.max_y[missing_cluster_p], name = 'Missing Cluster Peaks', showlegend=True))

fig_c_vs_s.add_trace(go.Scatter(mode='markers',marker=dict(size=7,color='#00FCF5'),x = cluster_sample.pks_time[missing_pt_p] , y= cluster_sample.pks_values[missing_pt_p], 
                          name = 'Missing Current Version Peaks', showlegend=True))


fig_c_vs_s.update_layout(xaxis = dict(tickmode = 'array', tickvals = cluster_sample.ticks_values, ticktext = cluster_sample.time_format_axis),autosize=False,width=1200,height=630,
                    title='Missing Cluster Peaks: '+str(len(missing_cluster_p))+'           Missing Current Version Peaks: '+str(len(missing_pt_p)) + 
                   '            MAW: '+str(sample_test.bs_r_window) + '         Band: '+str(sample_test.band_start)+' - '+str(sample_test.band_stop));


In [28]:
#fig_c_vs_s.show()

##

## Comparing different window lengths against 6 window length in mean average subtraction

In [29]:
#Visually missing peaks

fig_ms_p = go.Figure()
missing_6_dict ={};  missing_dif_wl_dict={};
spline=4;
res = 5000
lengths = np.arange(7,20,1)


for N in lengths: 
    sample_test_dif_wl.bs_r_window = N
    missing_6_p, missing_dif_wl_p = find_missing_peaks(sample_test.max_x, sample_test_dif_wl.max_x, True)
    missing_6_dict['window_length_'+str(N)] = missing_6_p
    missing_dif_wl_dict['window_length_'+str(N)] = missing_dif_wl_p
    
    fig_ms_p.add_trace(go.Scatter(line= dict(color='blue'),x=sample_test.time, y=sample_test.values,name='Raw Data',showlegend=True, visible = False))
    

    fig_ms_p.add_trace(go.Scatter(mode='markers',marker=dict(size=12),x=sample_test.max_x, y= sample_test.max_y, name = '6 WL Peaks', 
                               showlegend=True, visible = False))
    fig_ms_p.add_trace(go.Scatter(mode='markers',marker=dict(size=10,color='#21B626'), x = sample_test_dif_wl.max_x, y = sample_test_dif_wl.max_y,
                               name = str(N)+' WL Peaks',showlegend=True, 
                              visible = False))
    fig_ms_p.add_trace(go.Scatter(mode='markers',marker=dict(size=7,color='orange'), x = sample_test_dif_wl.max_x[missing_6_p], 
                               y= sample_test_dif_wl.max_y[missing_6_p], name = '6 WL Missing Peaks', showlegend=True, visible = False))
    
    fig_ms_p.add_trace(go.Scatter(mode='markers',marker=dict(size=7,color='#00FCF5'),x = sample_test.max_x[missing_dif_wl_p] , y= sample_test.max_y[missing_dif_wl_p], 
                              name = str(N)+' WL Missing  Peaks', showlegend=True, visible = False))

fig_ms_p.data[5].visible = True

steps = []
start = 0
for i in range(0,len(lengths)):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig_ms_p.data)},
              {"title": "Peaks identified with window length "+str(lengths[i])+ '   Amount of 6 WL Missing Peaks: '+
               str(len(missing_6_dict['window_length_'+str(lengths[i])])) +'    Amount of ' + str(lengths[i]) + ' WL Missing Peaks: '+ 
               str(len(missing_dif_wl_dict['window_length_'+str(lengths[i])]))}],  # layout attribute
    )
    step["args"][0]["visible"][start] = True  # Toggle i'th trace to "visible"
    step["args"][0]["visible"][start+1] = True
    step["args"][0]["visible"][start+2] = True
    step["args"][0]["visible"][start+3] = True
    step["args"][0]["visible"][start+4] = True
    
    start = start+5
    steps.append(step)
    
sliders = [dict(active=5, currentvalue={"prefix": "n: "}, pad={"t": 50}, steps=steps)]
fig_ms_p.update_layout(sliders=sliders);
fig_ms_p.update_layout(xaxis_range=[79000,85000]);
fig_ms_p.update_layout(xaxis = dict(tickmode = 'array', tickvals = cluster_sample.ticks_values, ticktext = cluster_sample.time_format_axis),autosize=False,width=1200,height=600); 

/home/shared/braulio/ecg_algorithm/functions.py:232: RuntimeWarning:

invalid value encountered in sqrt

/home/shared/braulio/ecg_algorithm/functions.py:233: RuntimeWarning:

invalid value encountered in sqrt



In [30]:
#fig_ms_p.show()

## Pre Processed data with different window lengths

In [31]:

fig_prepro = go.Figure()
missing_6_dict ={};  missing_dif_wl_dict={};
spline=4;
res = 5000
lengths = np.arange(7,20,1)


for N in lengths: 
    sample_test_dif_wl.bs_r_window = N
    
    fig_prepro.add_trace(go.Scatter(x = sample_test_dif_wl.prepro_x , 
                              y= sample_test_dif_wl.prepro_data, name = str(N)+' WL Prepro Data', showlegend=True, visible = False))

fig_prepro.data[3].visible = True

steps = []
start = 0
for i in range(0,len(lengths)):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig_prepro.data)},
              {"title": 'Preprocessed data, window length: '+str(lengths[i])}],  # layout attribute
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)
    
sliders = [dict(active=5, currentvalue={"prefix": "n: "}, pad={"t": 50}, steps=steps)]
fig_prepro.update_layout(sliders=sliders);
fig_prepro.update_layout(xaxis = dict(tickmode = 'array', tickvals = cluster_sample.ticks_values, ticktext = cluster_sample.time_format_axis),autosize=False,width=1200,height=600); 

In [32]:
#fig_prepro.show()

## Bandpass 

In [33]:

figbp = go.Figure()
missing_6_dict ={};  missing_dif_wl_dict={};
spline=4;
res = 5000
lengths = np.arange(7,20,1)


for N in lengths: 
    sample_test_dif_wl.bs_r_window = N
    
    figbp.add_trace(go.Scatter(x = sample_test_dif_wl.bp_x , 
                              y= sample_test_dif_wl.bp_y, name = str(N)+' WL in MS, Bandpass', showlegend=True, visible = False))

figbp.data[3].visible = True

steps = []
start = 0
for i in range(0,len(lengths)):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(figbp.data)},
              {"title": 'Bandpass , window length in mean subtraction: '+str(lengths[i])}],  # layout attribute
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)
    
sliders = [dict(active=5, currentvalue={"prefix": "n: "}, pad={"t": 50}, steps=steps)]
figbp.update_layout(sliders=sliders);
figbp.update_layout(xaxis = dict(tickmode = 'array', tickvals = cluster_sample.ticks_values, ticktext = cluster_sample.time_format_axis),autosize=False,width=1200,height=600); 

In [34]:
#figbp.show()

## Derivative

In [35]:

fig_der = go.Figure()
missing_6_dict ={};
spline=4;
res = 5000
lengths = np.arange(7,20,1)


for N in lengths: 
    sample_test_dif_wl.bs_r_window = N
    
    fig_der.add_trace(go.Scatter(x = sample_test_dif_wl.der_x , 
                              y= sample_test_dif_wl.der_y, name = str(N)+' WL in MS, Derivative', showlegend=True, visible = False))

fig_der.data[3].visible = True

steps = []
start = 0
for i in range(0,len(lengths)):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig_der.data)},
              {"title": 'Derivative , window length in mean subtraction: '+str(lengths[i])}],  # layout attribute
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)
    
sliders = [dict(active=5, currentvalue={"prefix": "n: "}, pad={"t": 50}, steps=steps)]
fig_der.update_layout(sliders=sliders);
fig_der.update_layout(xaxis = dict(tickmode = 'array', tickvals = sample_test.ticks_values, ticktext = sample_test.time_format_axis),autosize=False,width=1200,height=600); 

In [36]:
#fig_der.show()

## Moving Window Integration + Fiducial Points

In [37]:
mwi_fp = go.Figure()
missing_6_dict ={};  missing_dif_wl_dict={};
spline=4;
res = 5000
lengths = np.arange(7,20,1)


for N in lengths: 
    sample_test_dif_wl.bs_r_window = N
    
    mwi_fp.add_trace(go.Scatter(x = sample_test_dif_wl.mwi_x , 
                              y= sample_test_dif_wl.mwi_y, name = str(N)+' Bandpass WL: '+str(N), showlegend=True, visible = False))
    mwi_fp.add_trace(go.Scatter(mode = 'markers', x = sample_test_dif_wl.mwi_x[sample_test_dif_wl.fiducial_points] , 
                              y= sample_test_dif_wl.mwi_y[sample_test_dif_wl.fiducial_points], name = str(N)+' Bandpass WL: '+str(N), 
                              showlegend=True, visible = False))

mwi_fp.data[3].visible = True

steps = []
start = 0
for i in range(0,len(lengths)):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(mwi_fp.data)},
              {"title": 'Bandpass data, window length: '+str(lengths[i])}],  # layout attribute
    )
    step["args"][0]["visible"][start] = True  # Toggle i'th trace to "visible"
    step["args"][0]["visible"][start + 1] = True  # Toggle i'th trace to "visible"
    start = start + 2
    steps.append(step)
    
sliders = [dict(active=5, currentvalue={"prefix": "n: "}, pad={"t": 50}, steps=steps)]
mwi_fp.update_layout(sliders=sliders);
mwi_fp.update_layout(xaxis = dict(tickmode = 'array', tickvals = cluster_sample.ticks_values, ticktext = cluster_sample.time_format_axis),autosize=False,width=1200,height=600); 

In [38]:
#mwi_fp.show()

In [39]:
min_min_t = 1; min_sec_t = 30; max_min_t= 3; max_sec_t = 0; seconds_per_label_t = 1; inverted_t = False;   bs_r_window_t = 15; 
band_start_t = 3; band_stop_t = 100; fs_t= 250;  trans_width_t = 2.88; numtaps_t = 152; show_legend_t = False; mwi_window_t = 25 ;
test_segments_t =3;            spline_t = 4;       res_t = 5000;

recreated_cluster = sample_all_included(min_min_t,min_sec_t,max_min_t,max_sec_t,seconds_per_label_t,inverted_t, bs_r_window_t,band_start_t,band_stop_t,fs_t,
                       trans_width_t,numtaps_t, mwi_window_t, test_segments_t, spline_t,res_t)

### Recreating cluster sample 

In [40]:
comp_prepro = make_subplots(rows=2, cols=1, subplot_titles=("Cluster ", 'MWI: '+str(sample_test.bs_r_window)))
comp_prepro.add_trace(go.Scatter(x = recreated_cluster.prepro_x,y = recreated_cluster.prepro_data,showlegend=False),row=1, col=1)
comp_prepro.add_trace(go.Scatter(x = sample_test.prepro_x,y = sample_test.prepro_data,showlegend=False),row=2, col=1)

comp_prepro.update_layout(xaxis = dict(tickmode = 'array', tickvals = sample_test.ticks_values, ticktext = sample_test.time_format_axis),
                          xaxis2 = dict(tickmode = 'array', tickvals = recreated_cluster.ticks_values, ticktext = recreated_cluster.time_format_axis))
comp_prepro.update_layout(yaxis_range=[-1.2,1.2],yaxis2_range=[-0.45,0.4])
comp_prepro.update_layout(height=900,width = 1200);

In [41]:
sample_test.band=[4,100]
missing_pt_p, missing_cluster_p = find_missing_peaks(sample_test.max_x, cluster_sample.pks_time, True)
comp_bandpass = make_subplots(rows=3, cols=1, subplot_titles=('Peaks','MWI: '+str(sample_test.bs_r_window) , 'Cluster'))
comp_bandpass.add_trace(go.Scatter(x=cluster_sample.time, y= cluster_sample.values, name = 'Cluster', showlegend=True,),row = 1, col=1)
comp_bandpass.add_trace(go.Scatter(mode='markers',marker=dict(size=12),x=cluster_sample.pks_time, y= cluster_sample.pks_values, name = 'Cluster', 
                           showlegend=True,),row = 1, col=1)

comp_bandpass.add_trace(go.Scatter(mode='markers',marker=dict(size=10,color='#21B626'), x = sample_test.max_x, 
                          y = sample_test.max_y, name = 'Current Version', showlegend=True),row = 1, col=1)

comp_bandpass.add_trace(go.Scatter(mode='markers',marker=dict(size=7,color='orange'), x = sample_test.max_x[missing_cluster_p], 
                           y= sample_test.max_y[missing_cluster_p], name = 'Missing Cluster Peaks', showlegend=True),row = 1, col=1)

comp_bandpass.add_trace(go.Scatter(mode='markers',marker=dict(size=7,color='#00FCF5'),x = cluster_sample.pks_time[missing_pt_p] , y= cluster_sample.pks_values[missing_pt_p], 
                          name = 'Missing Current Version Peaks', showlegend=True),row = 1, col=1)

comp_bandpass.add_trace(go.Scatter(x = sample_test.bp_x,y = sample_test.bp_y,showlegend=False),row=2, col=1)
comp_bandpass.add_trace(go.Scatter(x = recreated_cluster.bp_x,y = recreated_cluster.bp_y,showlegend=False),row=3, col=1)

comp_bandpass.update_layout(xaxis = dict(tickmode = 'array', tickvals = cluster_sample.ticks_values, ticktext = cluster_sample.time_format_axis),
                            xaxis2 = dict(tickmode = 'array', tickvals = sample_test.ticks_values, ticktext = sample_test.time_format_axis),
                            xaxis3 = dict(tickmode = 'array', tickvals = recreated_cluster.ticks_values, ticktext = recreated_cluster.time_format_axis))
comp_bandpass.update_layout(xaxis_range=[141300,142900],xaxis2_range=[141300,142900],yaxis_range=[-2.8,3],yaxis2_range=[-0.2,0.2])
comp_bandpass.update_layout(height=1200,width = 1200);